In [1]:
import requests
import pandas as pd
import random

r = requests.get("https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations/stations.txt")
r.raise_for_status()
stations = r.text.strip().split("\n")
stations = random.sample(stations, k=10)
workload = random.choices(stations, k=100, weights=[0.3, 0.2] + [0.5/8]*8)

#df = pd.read_csv(f"https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations/{station}.csv.gz",
 #                        names=["station", "date", "element", "value", "m", "q", "s", "obs"])

workload[:10]

['US1WIWK0118',
 'US1WILC0025',
 'US1WIWK0118',
 'USC00470885',
 'US1WIWK0118',
 'US1WIMW0082',
 'US1WIKN0013',
 'US1WIWK0118',
 'US1WILC0025',
 'US1WIWK0118']

In [14]:
import time

cache_size = 4
cache = {} # key is the station, value in the DF
evict_order = [] # evict from front

#stats
hits = [] #1 is hit, 0 is miss
latencies = []

def get_df(station):
    start = time.time()
    if station in cache:
        print(f"hit {station}")
        
        hits.append(1)
        #LRU updates evict order when something used
        print(evict_order)
        evict_order.remove(station)
        evict_order.append(station)
        df = cache[station]
        #print(cache)
    else:
       # print("miss")
        hits.append(0)
        df = pd.read_csv(f"https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations/{station}.csv.gz",
                         names=["station", "date", "element", "value", "m", "q", "s", "obs"])
        cache[station] = df
        evict_order.append(station)
        
        if len(cache) > cache_size:
            #print('evict')
            victim = evict_order.pop(0)
            cache.pop(victim)
    #print(evict_order)
    end = time.time()
    latencies.append((end-start)*1000)
    #return df


In [15]:
for station in workload[:10]:
    df = get_df(station)
hit_rate = sum(hits) /len(hits)
print('Avg hit rate: ',hit_rate)
lat = sum(latencies) /len(latencies)
print('Avg latency: ',lat)

hit US1WIWK0118
['US1WIWK0118', 'US1WILC0025']
hit US1WIWK0118
['US1WILC0025', 'US1WIWK0118', 'USC00470885']
hit US1WIWK0118
['USC00470885', 'US1WIWK0118', 'US1WIMW0082', 'US1WIKN0013']
hit US1WIWK0118
['US1WIMW0082', 'US1WIKN0013', 'US1WIWK0118', 'US1WILC0025']
Avg hit rate:  0.4
Avg latency:  62.64212131500244


In [26]:
import numpy as np
print('tail latency:',np.quantile(latencies,.9))

tail latency: 127.18074321746826
